In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from selenium.common.exceptions import TimeoutException
import random

In [ ]:
service = Service(executable_path=r'/usr/bin/chromedriver')
options = webdriver.ChromeOptions()
#options.add_argument('--headless') #It allows users to run automated scripts in headless mode, meaning that the browser window wouldn’t be visible. 
#options.add_argument('--no-sandbox')
#options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=service, options=options)

url = 'https://realt.by/belarus/sale/dachi/?page=1'#"https://realt.by/sale/dachi/?page=1"
headers = ({'User-Agent':'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:124.0) Gecko/20100101 Firefox/124.0', 'Accept-Language':'en-US, en;q=0.5'})
driver.get(url);
time.sleep(5)

#try:
#    cookie_ok_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "cookie-banner__container"))) # 'cookie-banner__container'
#    cookie_ok_button.accept()
#except TimeoutException:
#    print("no cookie popup")

SCROLL_PAUSE_TIME = random.randint(5,8)

full_height = driver.execute_script("return document.body.scrollHeight")
height = 0 
cnt = 0
res = []

while True:
    res = []
    rand = random.randint(5,8)
    print(cnt)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    r = soup.find_all("div", class_="sm:w-full w-full p-1.5 sm:p-2.5")
    for e in r:
        add = e.find('a')['href']
        if add not in res:
            res.append(add)
    
    with open('data/pars_res.txt', 'a') as f:
        f.write(','.join(res)+',') 

    # Calculate new scroll height and compare with last scroll height
    new_height = height + driver.execute_script("return document.body.clientHeight")
    try:
        if new_height >= full_height and driver.find_element(By.LINK_TEXT, 'Далее'):
            #print('next is')
            driver.find_element(By.LINK_TEXT, 'Далее').click()
            time.sleep(rand)
            full_height = driver.execute_script("return document.body.scrollHeight")
            new_height = 0
            cnt +=1
    except:
        print('last')
        break    
    height = new_height 
    driver.execute_script(f"window.scrollTo(0, {height});")
    time.sleep(rand)
    

driver.quit()


In [2]:
with open('data/pars_res.txt', 'r') as f:
    res = f.readlines()

res = res[0].split(',')

res[0:5]

['/brest-region/sale-dachi/object/3227872/',
 '/brest-region/sale-dachi/object/3226705/',
 '/brest-region/sale-dachi/object/3292562/',
 '/brest-region/sale-dachi/object/3273030/',
 '/brest-region/sale-dachi/object/3290219/']

In [5]:
df = pd.DataFrame()
no_price = []

def get_item(url):
    url = 'https://realt.by' + url
    page = requests.get(url)
    time.sleep(3)
    if page.status_code != 200:
        print(url, page.status_code)
        return 0
    
    soup = BeautifulSoup(page.content, "html.parser")
    rdict = {'index': int(url.split('/')[-2])}
    r = soup.find_all('section', class_='bg-white flex flex-wrap md:p-6 my-4 rounded-md') 
    
    sections = {'Параметры объекта':10, 'Инфраструктура':10, 'Местоположение':10}
    for i,k in enumerate(r):
        if k.find('h3').text in sections:
            sections[k.find('h3').text] = i 

    try:
        res_1 = r[sections['Параметры объекта']].find("ul", class_="w-full -my-1").find_all('li', class_='relative py-1')
        for li in res_1:
            key = li.find('span', class_='text-basic sm:flex-shrink-0 mr-2')
            value = li.find('p').getText()
            if key.text not in rdict:
                rdict[key.text] = [value.split()]
    except:
        pass

    try:
        res_2 = r[sections['Инфраструктура']].find_all('p')
        for p in res_2:
            key = p.getText()
            rdict[key] = 1
    except:
        pass

    try:
        res_3 = r[sections['Местоположение']].find('ul', class_='w-full mb-0.5 -my-1')
        for li in res_3:
            key = li.find('span', class_='text-basic sm:flex-shrink-0 mr-2')
            if li.find('a'):
                value = li.find('a').contents[0]
            elif li.find('p'):
                value = li.find('p').getText()
            rdict[key.text] = [value.split()]
    except:
        pass

    try:
        res_4 = soup.find('div', class_='bg-white flex flex-wrap md:p-6 rounded-md').find('h2') 
        rdict['Цена'] = res_4.text.strip()
    except:
        print(url,'no price')
        no_price.append(url)
        pass
    return pd.DataFrame(rdict)


In [ ]:
for i in list(set(res)):
    try:
        df = pd.concat([df,get_item(i)], axis=0)
        df.to_csv('data/country_houses_2.csv')
    except:
        print(f'exception for {i}')
        continue